<a href="https://colab.research.google.com/github/iovanalex/codeSnippets/blob/master/Youtube_Comment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install emoji
!pip install vaderSentiment
!pip install google-api-python-client
#!pip install boto3 --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.4 MB/s eta 0:00:00


In [38]:
# For Fetching Comments
from googleapiclient.discovery import build
# For filtering comments
import re
# For filtering comments with just emojis
import emoji
# Analyze the sentiments of the comment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# For visualization
import matplotlib.pyplot as plt

from google.colab import userdata

import datetime

from google.colab import drive
drive.mount('/content/gdrive')

# Make sure to put in the API KEY as it won't work otherwise
API_KEY = userdata.get('yt_api_key');

youtube = build('youtube', 'v3', developerKey=API_KEY) # initializing Youtube API


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [44]:
def getCommentsByVideoId(video_id):

    video_response = youtube.videos().list(
      part='snippet',
      id=video_id
    ).execute()

    # Splitting the response for channelID
    video_snippet = video_response['items'][0]['snippet']
    uploader_channel_id = video_snippet['channelId']
    print("channel id: " + uploader_channel_id)

    timestamp=datetime.datetime.now().strftime("%Y%m%d%H%M%S");
    fileName="/content/gdrive/MyDrive/yt_observatory/"+"yt_"+video_id+"_"+timestamp+".csv"
    outFile = open(fileName, 'w', encoding='utf-8')

    print("Fetching Comments...")

    nextPageToken = None
    comments = []
    while len(comments) < 60000:
        request = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100,  # You can fetch up to 100 comments per request
            pageToken=nextPageToken
        )
        response = request.execute()
        for item in response['items']:

            comment = item['snippet']['topLevelComment']['snippet']

            textDisplay=comment['textDisplay']
            authorDisplayName=comment['authorDisplayName']
            publishedAt=comment['publishedAt']
            updatedAt=comment['updatedAt']
            authorChannelId=comment['authorChannelId']['value']
            likeCount=comment['likeCount']
            oneLine=uploader_channel_id+","+video_id+",\""+textDisplay+"\","+authorDisplayName+","+publishedAt+","+updatedAt+","+authorChannelId+","+str(likeCount)
            outFile.write(oneLine+"\n")
            #print(oneLine)


        nextPageToken = response.get('nextPageToken')
        #print(f"Fetched {len(comments)} comments.");
        if not nextPageToken:
            break

    outFile.close()
    print("Comments stored successfully in"+fileName)


In [76]:
#getCommentsByVideoId("eIyg7G8e3aA") #puscifer
#getCommentsByVideoId("Z2qAVs2HJGc") #ukraine war stories
#getCommentsByVideoId("4P5aC0dRppo") #GC 1 dec
#getCommentsByVideoId("lctvzaN7j1s") #GC 26 nov
#getCommentsByVideoId("PxiKSjq7hlo") #GC la Mandachi
getCommentsByVideoId("lKbiTDXwpY0") #experiment

channel id: UCrJdCKIddUxHExmbMN6gZ9A
Fetching Comments...
UCrJdCKIddUxHExmbMN6gZ9A,lKbiTDXwpY0,"the ad lowkey killed the vibe but money is money ig",@dergaika,2024-12-03T20:55:39Z,2024-12-03T20:55:39Z,UCKV_NxXnlBN8X_Yz3z6-pJw,0
UCrJdCKIddUxHExmbMN6gZ9A,lKbiTDXwpY0,"And coyote hunting with a 308 and 30-06? Silly.",@calvindirette4860,2024-12-03T20:35:58Z,2024-12-03T20:35:58Z,UCzDRifVBINj_jJxGcw1AKHw,0
UCrJdCKIddUxHExmbMN6gZ9A,lKbiTDXwpY0,"A cub coyote? No. A pup.",@calvindirette4860,2024-12-03T20:33:44Z,2024-12-03T20:33:44Z,UCzDRifVBINj_jJxGcw1AKHw,0
UCrJdCKIddUxHExmbMN6gZ9A,lKbiTDXwpY0,"It&#39;s pronounced HOLLER!!! lol",@wvmountaineer69,2024-12-03T17:57:25Z,2024-12-03T17:57:25Z,UCaVWbY6lDnIrrobYm_ZYR0A,0
UCrJdCKIddUxHExmbMN6gZ9A,lKbiTDXwpY0,"My tribe calls them the kings of the woods. We give offerings like coins or tobacco after successful hunts.",@lh3779,2024-12-03T17:48:33Z,2024-12-03T17:48:33Z,UCIw_-8MZ81OgYy81a0a72-g,0
UCrJdCKIddUxHExmbMN6gZ9A,lKbiTDXwpY0,"Tsul Kalu.",@CptZatōichi

In [70]:
def getVideoDetailsById(video_id):
  #video_id="eIyg7G8e3aA"
  video_response = youtube.videos().list(
        part='snippet',
        id=video_id
      ).execute()
  #print(video_response)
  etag=video_response['etag']
  id=video_response['items'][0]['id']
  publishedAt=video_response['items'][0]['snippet']['publishedAt']
  title=video_response['items'][0]['snippet']['title']
  description=video_response['items'][0]['snippet']['description']
  channelTitle=video_response['items'][0]['snippet']['channelTitle']
  channelId=video_response['items'][0]['snippet']['channelId'];
  tags=""
  if('tags' in video_response['items'][0]['snippet']):
    tags=video_response['items'][0]['snippet']['tags']
  categoryId=video_response['items'][0]['snippet']['categoryId']
  liveBroadcastContent=video_response['items'][0]['snippet']['liveBroadcastContent']
  defaultAudioLanguage=video_response['items'][0]['snippet']['defaultAudioLanguage']


  print(channelId+", "+id+", "+etag+", "+title+", "+publishedAt+", "+channelTitle+", "+categoryId+", "+liveBroadcastContent+", "+defaultAudioLanguage+", "+"\""+description+"\"");
  print(tags);


getVideoDetailsById("Z2qAVs2HJGc");


UCrJdCKIddUxHExmbMN6gZ9A, Z2qAVs2HJGc, KnlhtmyO1RFe0iqPW37AxRgvPBs, What Made Them Scream Like That..., 2022-03-03T06:34:45Z, Wartime Stories, 22, none, en, "ABOUT THE CHANNEL 
For all things strange and unusual relating to war and the military, Wartime Stories covers the most chilling cases of crimes against humanity, bizarre tales, unsolved mysteries, espionage, deception, creepy encounters of the paranormal, as well as feats of valor and ultimate sacrifice.

PATREON
https://www.patreon.com/wartimestories

OFFICIAL WEBSITE
Merchandise, Episode Suggestions, and more…
https://www.wartimestories.com

EMAIL 
info@wartimestories.com


MUSIC
“Tensions”, “Evil Rising”, “The Pain That Never Left”, and “Sneaky Action” by David Fesliyan, courtesy of Fesliyan Studios
https://www.FesliyanStudios.com

“Necrophilia” and “Lying in Wait” used by kind permission of CO.AG 
https://www.youtube.com/channel/UCcavSftXHgxLBWwLDm_bNvA

“Echoes of Time” and “Crypto” by Kevin MacLeod (incompetech.com) License

In [75]:
from googleapiclient.discovery import build

# Replace with your API key

YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

def get_videos_from_channel(channel_id):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)

    # Step 1: Get the Uploads playlist ID for the channel
    channel_response = youtube.channels().list(
        part='contentDetails',
        id=channel_id
    ).execute()

    if not channel_response['items']:
        print("Channel not found or no content available.")
        return []

    uploads_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    # Step 2: Fetch videos from the uploads playlist
    videos = []
    next_page_token = None
    while True:
        playlist_response = youtube.playlistItems().list(
            part='snippet',
            playlistId=uploads_playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        for item in playlist_response['items']:
            video_data = {
                'title': item['snippet']['title'],
                'videoId': item['snippet']['resourceId']['videoId'],
                'publishedAt': item['snippet']['publishedAt']
            }
            videos.append(video_data)

        next_page_token = playlist_response.get('nextPageToken')
        if not next_page_token:
            break

    return videos

# Example usage
channel_id = 'UCrJdCKIddUxHExmbMN6gZ9A'  # Replace with the desired channel ID
videos = get_videos_from_channel(channel_id)
for video in videos:
    print(f"Title: {video['title']}, Video ID: {video['videoId']}, Published At: {video['publishedAt']}")


Title: These Things Can Mimic Other Animals..., Video ID: lKbiTDXwpY0, Published At: 2024-11-22T17:10:56Z
Title: Why Would A Lieutenant Remove All of His Own Teeth..., Video ID: VJGoburWIJ4, Published At: 2024-11-12T22:03:55Z
Title: This Cemented My Belief in the Paranormal…, Video ID: tcr74RK7qCQ, Published At: 2024-10-25T16:35:24Z
Title: He Never Thought Skinwalkers Were Real..., Video ID: -B8YG1GNM7o, Published At: 2024-10-11T20:37:48Z
Title: If These Soldier’s Stories are True... Burn the Jungles, Video ID: 7u7E3WpYyYU, Published At: 2024-09-30T04:30:55Z
Title: USN Petty Officer and Others Report Megalodon Sightings, Video ID: 76RqtQyRrrk, Published At: 2024-09-14T07:32:47Z
Title: Men Who Everyone Thought Were Dead... Came Back, Video ID: 05o2sNF3vgQ, Published At: 2024-08-31T14:44:02Z
Title: What Killed These Irish and British Men?, Video ID: YZwHSi59ecA, Published At: 2024-08-10T13:00:36Z
Title: Shocking Stories from Soldiers and Drone Operators, Video ID: QkRRERFPfEM, Published 